In [1]:
%run "..\..\Startup_py3.py"
sys.path.append(r"..\..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

# library design specific tools
from ImageAnalysis3.library_tools import LibraryDesigner as ld
from ImageAnalysis3.library_tools import LibraryTools as lt
# biopython imports
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

22548


In [4]:
from ImageAnalysis3 import library_tools
pool_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-15_human_brain'
library_folder = os.path.join(pool_folder, f'human_genome_library')
report_folder = os.path.join(library_folder, 'reports')

probe_savefile = os.path.join(report_folder, f'merged_probes.pbr')


In [5]:
# Parse segemented list
_pbde = library_tools.design.pb_reports_class()
_pbde.load_from_file(probe_savefile)

- Loading from savefile: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-15_human_brain\human_genome_library\reports\merged_probes.pbr.
- loading cand_probes from file
- loading kept_probes from file
- 223012 filtered probes loaded
- loading sequence_dic from file
- loading map_dic from file
- loading params_dic from file
- loading check_dic from file
- loading save_file from file


True

In [6]:
# save to csv
cand_probe_dict = {_s:_v for _s,_v in _pbde.cand_probes.items() if _v['reg_index']==0}

In [8]:
    check_dic={('genome','self_sequences'): 25,
               'rep_genome': 0,
               'transcriptome': 25, # 14
               #'ref_merfish': 14,
               'gc':[0.25,0.75],'tm': 37+0.62*50+5,
               }

In [10]:
from ImageAnalysis3.library_tools.LibraryTools import fastaread

In [12]:
names, seqs = fastaread(r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-15_human_brain\human_genome_library\sequences\all_reg_1.fasta')

In [23]:
kept_cand_probes = check_region_probes(names[0], seqs[0], cand_probe_dict, check_dic,)

-- checking 37596 candidate probes in region: 1:1240000-1260000_:+
- 11908 probes pass quality checks. 


In [ ]:
# optimize coverage:


In [24]:
def check_region_probes(
    _reg_name, _reg_seq,
    _cand_probe_dict, _check_dic, 
    buffer_len=0, 
    pick_probe_by_hit=True, verbose=True):
    
    _filtered_probe_dict = {}
    
    if verbose:
        print(f"-- checking {len(_cand_probe_dict)} candidate probes in region: {_reg_name}")
    # loop through probes
    for _pb, _info in _cand_probe_dict.items():
        # 0. remove edge probes
        _edge_size = int(max(buffer_len, 0))
        if _info['pb_index'] < _edge_size or _info['pb_index'] > len(_reg_seq)-len(_pb)+1-_edge_size:
            continue
        _check_gc, _check_tm = True, True
        _check_mask = True
        _check_map = True
        # 1. discard if GC or Tm doesn't pass the threshold:
        if 'gc' in  _check_dic:
            if isinstance(_check_dic['gc'], list) or isinstance(_check_dic['gc'], tuple):
                if _info['gc'] > np.max(_check_dic['gc']) or _info['gc'] < np.min(_check_dic['gc']):
                    _check_gc = False
            else:
                if _info['gc'] < _check_dic['gc']:
                    _check_gc = False
        if 'tm' in _check_dic:
            if isinstance(_check_dic['tm'], list) or isinstance(_check_dic['tm'], tuple):
                if _info['tm'] > np.max(_check_dic['tm']) or _info['tm'] < np.min(_check_dic['tm']):
                    _check_tm = False
            else:
                if _info['tm'] < _check_dic['tm']:
                    _check_tm = False
        if not _check_gc or not _check_tm:
            continue
        # 2. masking privided sequences
        _mask_check = True
        if 'masks' in _check_dic:
            # get existence
            _mask_exists = [find_sub_sequence(_pb, _str) for _str in _check_dic['masks']]
            # skip this probe if masked
            if np.sum(_mask_exists) > 0:
                _check_mask = False
        if not _check_mask:
            continue
        # 3. calculate map values
        _map_score_dict = {}
        for _check_key, _thres in _check_dic.items(): # loop through 
            if _check_key not in ['gc', 'tm', 'masks']:
                if isinstance(_check_key, str):
                    _check_map_key = f"map_{_check_key}"
                    _pb_map_value = _info[_check_map_key]

                    if _pb_map_value > _thres:
                        _check_map = False
                        break 
                    else:
                        # zero hit with non-zero threshold
                        if _pb_map_value <= 0 and _thres > 0:
                            _map_score_dict[_check_key] = _thres / 0.5 #
                        # if both zero hits and zero threshold ,skip
                        elif _pb_map_value <= 0 and _thres <= 0: 
                            _map_score_dict[_check_key] = np.nan 
                        # have hits with positive threshold
                        else:    
                            _map_score_dict[_check_key] =  _thres / _pb_map_value

                elif isinstance(_check_key, list) or isinstance(_check_key, tuple):
                    _tar_map_key, _ref_map_key = f"map_{_check_key[0]}", f"map_{_check_key[1]}"
                    _pb_map_value = _info[_tar_map_key] - _info[_ref_map_key]

                    if _pb_map_value > _thres:
                        _check_map = False
                        break 
                    else:
                        # zero hit with non-zero threshold
                        if _pb_map_value <= 0 and _thres > 0:
                            _map_score_dict[_check_key] = _thres / 0.5 #
                        # if both zero hits and zero threshold ,skip
                        elif _pb_map_value <= 0 and _thres <= 0: 
                            _map_score_dict[_check_key] = np.nan 
                        # have hits with positive threshold
                        else:    
                            _map_score_dict[_check_key] =  _thres / _pb_map_value
        if not _check_map:
            continue
        else:
            pass
        # if everything passed, store it temporarily
        if _check_gc and _check_tm and _check_mask and _check_map:
            _filtered_probe_dict[_pb] = _info
    if verbose:
        print(f"- {len(_filtered_probe_dict)} probes passed quality checks. ")
        # 4. 
    return _filtered_probe_dict